In [35]:
from elasticsearch import Elasticsearch
from transformers import BertTokenizer, BertModel
import torch
import json

es = Elasticsearch(
    cloud_id="8c25b2c39346426e8cb23b57abe7fdc7:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDVjNjdhZjUyNGZhYTQzNTlhNGJjOTlkMjMxMWI2NjczJGFjMDYyMDg3NDBlODRlMjJiMDMyYzkxM2FjNTJjOGY5",
    api_key=('N2duOEI0b0JkSThuM0lleVJid0o6QkllTXA5bjFSYVdmZzBla0xEWXVTQQ==')
)

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        
    return embeddings.numpy()[0] 

In [37]:
with open("C:/Users/Mourad/Documents/Git/akk_search_tool/test.json", "r", encoding="utf-8") as file:
    documents_list = json.load(file)

In [38]:
def process_and_index_documents(documents):
    for doc in documents:
        # Check if not empty
        title_content = doc.get("first_title", "") + " " + doc.get("main_title", "")
        if title_content.strip():  
            doc["title_embedding"] = get_embedding(title_content).tolist()

        text_content_fr = doc.get("fr_text", "")
        if text_content_fr.strip():  
            doc["fr_text_embedding"] = get_embedding(text_content_fr).tolist()

        text_content_nl = doc.get("nl_text", "")
        if text_content_nl.strip():  
            doc["nl_text_embedding"] = get_embedding(text_content_nl).tolist()

        author_content = doc.get("Auteur(s)", "")
        if author_content.strip():  
            doc["author_embedding"] = get_embedding(author_content).tolist()

        # Index the document with embeddings into Elasticsearch
        es.index(index="documents", body=doc)

process_and_index_documents(documents_list)

C:\Users\Mourad\AppData\Local\Temp\ipykernel_1152\1312917292.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index="documents", body=doc)


In [39]:
with open("C:/Users/Mourad/Documents/Git/akk_search_tool/test.json", "w", encoding="utf-8") as file:
    json.dump(documents_list, file, ensure_ascii=False, indent=4)

In [41]:
def semantic_search(query_text, fields=["title_embedding"], min_score=0.25):
    query_embedding = get_embedding(query_text)
    
    should_queries = []
    for field in fields:
        should_queries.append({
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": f"cosineSimilarity(params.query_vector, '{field}') + 1.0",
                    "params": {"query_vector": query_embedding.tolist()}
                }
            }
        })
    
    search_query = {
        "query": {
            "bool": {
                "should": should_queries
            }
        },
        "min_score": min_score
    }
    
    response = es.search(index="documents", body=search_query)
    return response


query = "projet de loi"
fields_to_search = ["title_embedding", "fr_text_embedding", "nl_text_embedding"]

results = semantic_search(query, fields=fields_to_search, min_score=0.25)
top_hits = results['hits']['hits']

print("Top matched documents with a similarity score of at least 0.25:")
for hit in top_hits:
    print(hit["_source"]["content"]) 


C:\Users\Mourad\AppData\Local\Temp\ipykernel_1152\2563114661.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="documents", body=search_query)


BadRequestError: BadRequestError(400, 'search_phase_execution_exception', "class_cast_exception: class org.elasticsearch.script.field.FloatDocValuesField cannot be cast to class org.elasticsearch.script.field.vectors.DenseVectorDocValuesField (org.elasticsearch.script.field.FloatDocValuesField and org.elasticsearch.script.field.vectors.DenseVectorDocValuesField are in module org.elasticsearch.server@8.9.1 of loader 'app')")